In [545]:
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
from difflib import SequenceMatcher 

In [259]:
filepath = "/Users/liudmylaslava/Downloads/test.txt"
with open(filepath) as fp:
    content = fp.read()
content[:1000]

'S Keeping the Secret of Genetic Testing\n\nS What is genetic risk ?\n\nS Genetic risk refers more to your chance of inheriting a disorder or disease .\nA 3 4|||ArtOrDet||||||REQUIRED|||-NONE-|||0\n\nS People get certain disease because of genetic changes .\nA 3 4|||Nn|||diseases|||REQUIRED|||-NONE-|||0\n\nS How much a genetic change tells us about your chance of developing a disorder is not always clear .\n\nS If your genetic results indicate that you have gene changes associated with an increased risk of heart disease , it does not mean that you definitely will develop heart disease .\nA -1 -1|||noop|||-NONE-|||REQUIRED|||-NONE-|||0\nA 25 27|||WOadv|||will definitely|||REQUIRED|||-NONE-|||1\n\nS The opposite is also true .\n\nS If your genetic results show that you do not have changes associated with an increased risk of heart disease , it is still possible that you develop heart disease .\nA -1 -1|||noop|||-NONE-|||REQUIRED|||-NONE-|||0\nA 26 26|||Vt|||will|||REQUIRED|||-NONE-|||1\n

In [471]:
text = content.split('\n\n')
data = []
for t in text:
    if '|||' in t:
        chunk = t.split('\n')
        for c in chunk[1:]:
            tr = c+'|||'+chunk[0]
            res = tr.split('|||')
            data.append(res)
df = pd.DataFrame(data = data, columns=['position',  'error_type', 'edit', 'req', 'no', 'ann_id', 'sent'])

In [472]:
df.head()

,position,error_type,edit,req,no,ann_id,sent
0,A 3 4,ArtOrDet,,REQUIRED,-NONE-,0,S Genetic risk refers more to your chance of i...
1,A 3 4,Nn,diseases,REQUIRED,-NONE-,0,S People get certain disease because of geneti...
2,A -1 -1,noop,-NONE-,REQUIRED,-NONE-,0,S If your genetic results indicate that you ha...
3,A 25 27,WOadv,will definitely,REQUIRED,-NONE-,1,S If your genetic results indicate that you ha...
4,A -1 -1,noop,-NONE-,REQUIRED,-NONE-,0,S If your genetic results show that you do not...


In [473]:
# Delete a strange row from df
df = df.loc[~df['error_type'].str.startswith('S Social media')]
# Start with 0th and 1th annotators
df_chunk = df[df['ann_id'].isin(['1','0'])]
# how many annotations we have for each sentence 
max_ann = df_chunk.groupby(['sent', 'ann_id']).size().reset_index(name='counts').groupby(['sent'])['counts'].max().reset_index(name='max_ann')
df_chunk = pd.merge(df_chunk, max_ann, on='sent', how='left')
# Calculate total number of questions with 0th and 1th annotators
total = df_chunk.groupby(['sent'], as_index=False)['ann_id'].nunique().value_counts()[2]
print(f'Total number of qustions for 1th and 2th ann: {total}')
# cases where annotators agreed
full = df_chunk[df_chunk[['position', 'edit', 'sent']].duplicated(keep='first')]
result = sum(1/full['max_ann'])
print(f'Cases where they agreed: {result}')

Total number of qustions for 1th and 2th ann: 1128
Cases where they agreed: 213.6834596695644


In [367]:
# Deal with cases where position the same but edition is not
rest = df_chunk[~df_chunk[['position', 'edit', 'sent']].duplicated(keep=False)]
part = rest[rest[['position', 'sent']].duplicated(keep=False)]
p = part.groupby(['sent','position', 'max_ann'])['edit'].apply(lambda x: ','.join(x)).reset_index()
# split edit column into two columns 
p[['first', 'second']] = p['edit'].str.split(',', n=1, expand=True)
p.head()

,sent,position,max_ann,edit,first,second
0,S A research shows that people will unconsciou...,A 8 11,5,",their",,their
1,S A study of New York University in 2010 shown...,A 8 9,4,"shows,showed",shows,showed
2,"S Above all , life is more important than secr...",A 8 9,1,"secrets,secrecy",secrets,secrecy
3,S Advantages : Social media sites have greatly...,A 7 8,2,"eased,facilitated",eased,facilitated
4,S After realizing the he or she is tshe carrie...,A 1 2,3,"discovering,he or she realizes",discovering,he or she realizes


In [315]:
def longestSubstring(str1,str2):
    
    seqMatch = SequenceMatcher(None,str1,str2) 
  
    # find match of longest sub-string 
    match = seqMatch.find_longest_match(0, len(str1), 0, len(str2))
    # substring normalazing by dividing by longer string
    if len(str1)>=len(str2):
        res = match.size/len(str1)
    else:
        res = match.size/len(str2)
  
    return res

In [368]:
p['sim'] = p.loc[:, ["first",'second']].apply(lambda x: longestSubstring(*x), axis=1)
p.head()

,sent,position,max_ann,edit,first,second,sim
0,S A research shows that people will unconsciou...,A 8 11,5,",their",,their,0.000000
1,S A study of New York University in 2010 shown...,A 8 9,4,"shows,showed",shows,showed,0.666667
2,"S Above all , life is more important than secr...",A 8 9,1,"secrets,secrecy",secrets,secrecy,0.714286
3,S Advantages : Social media sites have greatly...,A 7 8,2,"eased,facilitated",eased,facilitated,0.181818
4,S After realizing the he or she is tshe carrie...,A 1 2,3,"discovering,he or she realizes",discovering,he or she realizes,0.055556


In [372]:
part_res = sum(p['sim']/p['max_ann'])
final = (result+part_res)/total
print(f"Annotator agreement between 0th and 1th = {final}")

Annotator agreement between 0th and 1th = 0.20664969207929745


In [547]:
def inner_agreement(str1, str2, df):
    df = df.loc[~df['error_type'].str.startswith('S Social media')]
    df_chunk = df[df['ann_id'].isin([str1, str2])]
    if len(df_chunk)==0:
        final = 'no'
    else:
        max_ann = df_chunk.groupby(['sent', 'ann_id']).size().reset_index(name='counts').groupby(['sent'])['counts'].max().reset_index(name='max_ann')
        df_chunk = pd.merge(df_chunk, max_ann, on='sent', how='left')
        if df_chunk.groupby(['sent'], as_index=False)['ann_id'].nunique().max()==1:
            final = 'no'
        else:
            total = df_chunk.groupby(['sent'], as_index=False)['ann_id'].nunique().value_counts()[2]
            full = df_chunk[df_chunk[['position', 'edit', 'sent']].duplicated(keep='first')]
            result = sum(1/full['max_ann'])
            rest = df_chunk[~df_chunk[['position', 'edit', 'sent']].duplicated(keep=False)]
            part = rest[rest[['position', 'sent']].duplicated(keep=False)]
            if len(part)!=0:
                p = part.groupby(['sent','position', 'max_ann'])['edit'].apply(lambda x: ','.join(x)).reset_index()
                p[['first', 'second']] = p['edit'].str.split(',', n=1, expand=True)
                p['sim'] = p.loc[:, ["first",'second']].apply(lambda x: longestSubstring(*x), axis=1)
                part_res = sum(p['sim']/p['max_ann'])
            else:
                part_res = 0
            final = (result+part_res)/total
    return final

In [523]:
# В цілому досить низька згода анотувальників
# Багато виправлень не збігається, бо у кожного анотувальника було своє уявлення як позначати виправлення 

# do the same for all pairs 
annot = [str(i) for i in list(range(0, 5))]
pairs = [[annot[a1], annot[a2]] for a1 in range(len(annot)) for a2 in range(a1+1,len(annot))]
res = 0
for pair in pairs:
    print(f'For pair: {pair} argeement = {inner_agreement(pair[0], pair[1], df)}')
    res += inner_agreement(pair[0], pair[1], df)
print(f'Total inter annotator agreement = {res/10}')

For pair: ['0', '1'] argeement = 0.20664969207929745
For pair: ['0', '2'] argeement = 0.2976265796241484
For pair: ['0', '3'] argeement = 0.37698635083667476
For pair: ['0', '4'] argeement = 0.5554843304843307
For pair: ['1', '2'] argeement = 0.36711648458266993
For pair: ['1', '3'] argeement = 0.4476024335695322
For pair: ['1', '4'] argeement = 0.4597222222222223
For pair: ['2', '3'] argeement = 0.4142490199693589
For pair: ['2', '4'] argeement = 0.39014041514041514
For pair: ['3', '4'] argeement = 0.42685185185185204
Total inter annotator agreement = 0.3942429380360502


In [ ]:
############################################

In [549]:
error_types = df['error_type'].unique().tolist()
for error in error_types:
        df_er = df[df['error_type']==error]
        result = []
        for pair in pairs:
            res = inner_agreement(pair[0], pair[1], df_er)
            result.append(res)
            agr = [r for r in result if not r=='no']
        if agr:
            print(f'Total inter annotator agreement for error type: {error} = {sum(agr)/len(agr)}')
                

Total inter annotator agreement for error type: ArtOrDet = 0.5996393012215809
Total inter annotator agreement for error type: Nn = 0.8888407551401251
Total inter annotator agreement for error type: WOadv = 0.9166666666666666
Total inter annotator agreement for error type: Vt = 0.8469644354003959
Total inter annotator agreement for error type: Trans = 0.7121396705146704
Total inter annotator agreement for error type: Mec = 0.8142847857434511
Total inter annotator agreement for error type: Prep = 0.7503576080016177
Total inter annotator agreement for error type: Rloc- = 0.7869312169312169
Total inter annotator agreement for error type: Wci = 0.6852135154503123
Total inter annotator agreement for error type: Wform = 0.7907510288065843
Total inter annotator agreement for error type: Vform = 0.8281413847590319
Total inter annotator agreement for error type: Vm = 0.5977272727272728
Total inter annotator agreement for error type: Others = 0.832096786957898
Total inter annotator agreement for 